# LLM RAG News Article QnA

In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install --upgrade pip -q
!pip install -q langchain-community
!pip install -q langchain-ollama
!pip install -q faiss-cpu
!curl -fsSL --no-progress-meter https://ollama.com/install.sh | sh
# What -fsSL even mean??? -> https://kodekloud.com/community/t/curl-fssl-command-meaning/74713

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [langchain-community]ngchain-community]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is i

## Import Libraries


In [3]:
# Kaggle run Ollama config only
import subprocess
process = subprocess.Popen('ollama serve; python3 python3 -c "import warnings; warnings.filterwarnings("ignore"); print("Done!")" ', shell=True) #runs on a different thread

In [4]:
# Imports
import ollama

from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

import pandas as pd
import time
from IPython.display import display, Markdown

import re

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAINl8USuxNWDLX6USuahDtbp5HDyV3cwrQzQRwFaQPt+V



time=2025-06-04T08:49:53.842Z level=INFO source=routes.go:1234 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICE

## Read data

In [5]:
data = pd.read_parquet('/kaggle/input/ds310-vietnamese-news-summary/train_data.parquet')
data.head(10)

,summary_id,article_id,summary,article,mint,lcsr
0,22887248,22884536,Không quân Mỹ ngày 30/7 cho biết 2 máy bay ném...,"Ngày 30-7, Mỹ đã điều động 2 máy bay ném bom s...",0.620571,3.400000
1,23299990,23286469,Hai quả tên lửa Hyunmoo-2 đã được Hàn Quốc phó...,Chỉ 6 phút sau khi Triều Tiên bắn thử tên lửa ...,0.443295,3.951220
2,21868543,21868825,Các công tố viên Hàn Quốc sẽ đưa ra đề nghị bắ...,"Ngày 27/3, các công tố viên Hàn Quốc đã xin lệ...",0.639676,3.490566
3,23803776,23805197,"Ngày 3-11, thông tin từ Đồn Biên phòng Cửa Đại...",Thông tin từ Đồn Biên phòng Cửa Đại (Bộ đội Bi...,0.303665,3.422222
4,21240367,21249367,Một trẻ 10 ngày tuổi ở Pháp đã tử vong sau khi...,Bộ Y tế Pháp ngày 4/1 công bố ngừng bán Vitami...,0.310787,3.956522
5,23451141,23453164,"Theo công ty mẹ của Yahoo, Verizon, vụ vi phạm...","Theo tiết lộ từ Verizon, hãng mua tài sản onli...",0.851073,3.351351
6,22342028,22342731,Truyền thông Nhật Bản đưa tin Chủ tịch Trung Q...,Truyền thông Nhật Bản ngày 22-5 đưa tin: Hồi t...,0.257584,4.489796
7,21662342,21657713,"""Tôi tin chắc rằng chuyến thăm Việt Nam của Nh...",Đây là chuyến thăm đầu tiên của Nhà vua và Hoà...,0.448921,3.819672
8,21311477,21311897,"Choi Soon-sil, nhân vật trung tâm của vụ bê bố...","Bạn thân của Tổng thống Hàn Quốc, bà Choi Soon...",0.225991,3.884615
9,21509222,21496906,Cơ quan tình báo Mỹ và Hàn Quốc cho rằng Triều...,Triều Tiên được cho là sở hữu lượng nguyên liệ...,0.599324,3.212766


In [6]:
article = str(data.iloc[8]['article'])
print(article)

Bạn thân của Tổng thống Hàn Quốc, bà Choi Soon-sil, thừa nhận đã đưa ra ý kiến chỉnh sửa đối với một số bài phát biểu kể từ khi bà Park Geun-hye còn là ứng cử viên tổng thống.

Hãng tin Yonhap dẫn Cơ quan Công tố Hàn Quốc cho biết bà Choi Soon-sil - nhân vật trung tâm của vụ bê bối chính trị hiện nay ở Hàn Quốc và là bạn thân lâu năm của Tổng thống Park Geun-hye, thừa nhận đã giúp bà Park chỉnh sửa nhiều bài phát biểu.
Phía công tố đã trình lời nhận tội của bà Choi ra trước Tòa án Hiến pháp trong buổi điều trần thứ hai, trong đó bà Choi thừa nhận đã đưa ra ý kiến chỉnh sửa đối với một số bài phát biểu kể từ khi bà Park Geun-hye còn là ứng cử viên tổng thống.
Một số ý kiến được chấp nhận và sử dụng. Tuy nhiên, bà Choi tuyên bố việc này là do bà hiểu rất rõ cách nghĩ của Tổng thống Park.
Bà Choi Soon-sil đã bị bắt hồi tháng 11/2016 với những cáo buộc can thiệp vào công việc nhà nước trong khi không giữ bất kỳ vị trí nào trong chính phủ, cũng như lợi dụng mối quan hệ cá nhân với Tổng thốn

## Chunking data

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
start = time.time()
chunks = text_splitter.split_text(text=article)
end = time.time() - start
print(f"Number of chunks : {len(chunks)}")
print(f"Chunking time: {end*1000} ms")

Number of chunks : 3
Chunking time: 0.3027915954589844 ms


In [8]:
for i in range(len(chunks)):
    print(f"Chunk {i}:\n", chunks[i], "\n\n")

Chunk 0:
 Bạn thân của Tổng thống Hàn Quốc, bà Choi Soon-sil, thừa nhận đã đưa ra ý kiến chỉnh sửa đối với một số bài phát biểu kể từ khi bà Park Geun-hye còn là ứng cử viên tổng thống. 


Chunk 1:
 Hãng tin Yonhap dẫn Cơ quan Công tố Hàn Quốc cho biết bà Choi Soon-sil - nhân vật trung tâm của vụ bê bối chính trị hiện nay ở Hàn Quốc và là bạn thân lâu năm của Tổng thống Park Geun-hye, thừa nhận đã giúp bà Park chỉnh sửa nhiều bài phát biểu.
Phía công tố đã trình lời nhận tội của bà Choi ra trước Tòa án Hiến pháp trong buổi điều trần thứ hai, trong đó bà Choi thừa nhận đã đưa ra ý kiến chỉnh sửa đối với một số bài phát biểu kể từ khi bà Park Geun-hye còn là ứng cử viên tổng thống. 


Chunk 2:
 Một số ý kiến được chấp nhận và sử dụng. Tuy nhiên, bà Choi tuyên bố việc này là do bà hiểu rất rõ cách nghĩ của Tổng thống Park.
Bà Choi Soon-sil đã bị bắt hồi tháng 11/2016 với những cáo buộc can thiệp vào công việc nhà nước trong khi không giữ bất kỳ vị trí nào trong chính phủ, cũng như lợi dụn

## Create vector database

In [9]:
def get_vectordb(text_chunks):
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    vectordb = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectordb

In [10]:
ollama.pull("nomic-embed-text")
vectordb = get_vectordb(chunks)

time=2025-06-04T08:49:57.045Z level=INFO source=download.go:177 msg="downloading 970aa74c0a90 in 3 100 MB part(s)"
time=2025-06-04T08:49:59.376Z level=INFO source=download.go:177 msg="downloading c71d239df917 in 1 11 KB part(s)"
time=2025-06-04T08:50:00.636Z level=INFO source=download.go:177 msg="downloading ce4a164fc046 in 1 17 B part(s)"
time=2025-06-04T08:50:01.889Z level=INFO source=download.go:177 msg="downloading 31df23ea7daa in 1 420 B part(s)"


[GIN] 2025/06/04 - 08:50:03 | 200 |  7.494783626s |       127.0.0.1 | POST     "/api/pull"


time=2025-06-04T08:50:03.911Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:03.912Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
time=2025-06-04T08:50:03.912Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:03.912Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:04.026Z level=INFO source=sched.go:788 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-970aa74c0a90ef7482477cf803618e776e173c007bf957f635f1015bfcfef0e6 gpu=GPU-56f1f503-81d0-2965-472b-0f6a79849008 parallel=1 available=16790978560 required="809.9 MiB"
time=2025-06-04T08:50:04.126Z level=INFO source=server.go:135 msg="system memory" total="31.4 GiB" free="29.9 GiB" free_swap="0 B"
time=2025-06-04T08:50:04.127Z level=INFO source=server.go:168 msg=offload library=cuda layers.requested=-

[GIN] 2025/06/04 - 08:50:04 | 200 |  885.221796ms |       127.0.0.1 | POST     "/api/embed"


time=2025-06-04T08:50:04.678Z level=INFO source=server.go:630 msg="llama runner started in 0.50 seconds"
decode: cannot decode batches with this context (use llama_encode() instead)
decode: cannot decode batches with this context (use llama_encode() instead)
decode: cannot decode batches with this context (use llama_encode() instead)


## Set up LLM and Retrieval

In [11]:
# Set up LLM and retrieval
ollama.pull("qwen3:1.7b")
local_model = "qwen3:1.7b"  
llm = ChatOllama(model=local_model, enable_thinking=False, Temperature=0.7, TopP=0.8, TopK=20, MinP=0)

time=2025-06-04T08:50:05.386Z level=INFO source=download.go:177 msg="downloading 3d0b790534fe in 14 100 MB part(s)"
time=2025-06-04T08:50:08.663Z level=INFO source=download.go:177 msg="downloading ae370d884f10 in 1 1.7 KB part(s)"
time=2025-06-04T08:50:09.913Z level=INFO source=download.go:177 msg="downloading d18a5cc71b84 in 1 11 KB part(s)"
time=2025-06-04T08:50:11.173Z level=INFO source=download.go:177 msg="downloading cff3f395ef37 in 1 120 B part(s)"
time=2025-06-04T08:50:12.453Z level=INFO source=download.go:177 msg="downloading 517ccaff02fe in 1 487 B part(s)"


[GIN] 2025/06/04 - 08:50:18 | 200 | 13.187129161s |       127.0.0.1 | POST     "/api/pull"


In [12]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""
    Bạn là một trợ lý với nhiệm vụ giúp đỡ người dùng trả lời những câu hỏi liên quan đến bài báo.
    Bạn sẽ thực hiện điều này bằng cách tạo ra thêm 2 câu hỏi khác tương đồng với câu hỏi gốc để 
    tím kiếm những thông tin liên quan trong Vector Database.
    Bằng cách tạo ra thêm những góc mới tương đồng với câu hỏi của người dùng, mục tiêu của bạn là 
    giúp người dùng vượt qua rào cản của việc tìm kiếm thông tin dựa trên khoảng cách tương đối về 
    mặt ngữ nghĩa của chúng. 
    Hãy cố gắng để không trả lời những thông tin tồn tại trong bài báo.
    Nếu bạn không biết câu trả lời, hãy cứ trả lời là không biết.
    Hãy đưa ra câu trả lời trên một dòng mới và đừng lặp lại câu hỏi của người dùng. 
    Hãy đưa ra câu trả lời một cách ngắn gọn.
    Câu hỏi: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vectordb.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT,
)

## Create chain

In [13]:
# RAG prompt template
template = """Hãy trả lời câu hỏi mà CHỈ dựa vào thông tin được cho như sau:
{context}
Câu hỏi: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## News Article QnA

In [15]:
def news_qa(question):
    return display(Markdown(re.sub('<think>(?s:.)*?</think>', '', chain.invoke(question))))

In [16]:
# Summarize the article
news_qa("Hãy tóm tắt bài báo này.")

time=2025-06-04T08:50:22.287Z level=INFO source=sched.go:548 msg="updated VRAM based on existing loaded models" gpu=GPU-56f1f503-81d0-2965-472b-0f6a79849008 library=cuda total="15.9 GiB" available="15.1 GiB"
time=2025-06-04T08:50:22.288Z level=INFO source=sched.go:788 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-3d0b790534fe4b79525fc3692950408dca41171676ed7e21db57af5c65ef6ab6 gpu=GPU-56f1f503-81d0-2965-472b-0f6a79849008 parallel=2 available=16210342912 required="2.8 GiB"
time=2025-06-04T08:50:22.394Z level=INFO source=server.go:135 msg="system memory" total="31.4 GiB" free="29.6 GiB" free_swap="0 B"
time=2025-06-04T08:50:22.395Z level=INFO source=server.go:168 msg=offload library=cuda layers.requested=-1 layers.model=29 layers.offload=29 layers.split="" memory.available="[15.1 GiB]" memory.gpu_overhead="0 B" memory.required.full="2.8 GiB" memory.required.partial="2.8 GiB" memory.required.kv="896.0 MiB" memory.required.allocat

[GIN] 2025/06/04 - 08:50:27 | 200 |  5.849706213s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/06/04 - 08:50:27 | 200 |   27.115521ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   27.697671ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   24.386276ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   27.596366ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   25.290362ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   27.152925ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   22.394134ms |       127.0.0.1 | POST     "/api/embed"


time=2025-06-04T08:50:27.974Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:27.974Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:27.974Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
time=2025-06-04T08:50:27.974Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
decode: cannot decode batches with this context (use llama_encode() instead)
time=2025-06-04T08:50:28.005Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:28.005Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:28.005Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:28.005Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
decode: cannot decode batches with this context (use llam

[GIN] 2025/06/04 - 08:50:28 | 200 |   24.252665ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   22.663472ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:28 | 200 |   21.837231ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:32 | 200 |  4.467291706s |       127.0.0.1 | POST     "/api/chat"




Bài báo tóm tắt rằng bà Choi Soon-sil, bạn thân của Tổng thống Hàn Quốc Park Geun-hye, bị điều tra về việc can thiệp vào công việc nhà nước và lợi dụng mối quan hệ cá nhân để gây quỹ bất hợp pháp. Bà đã bác bỏ cáo buộc, nhưng Cơ quan Công tố Hàn Quốc đã trình lời nhận tội của bà trước Tòa án Hiến pháp, xác nhận bà đã giúp Park chỉnh sửa nhiều bài phát biểu khi cô còn là ứng cử viên tổng thống. Vụ bê bối đang thu hút sự chú ý của dư luận.

In [17]:
# Ask about relevant information within article
news_qa("Sự kiện gì đã diễn ra vào tháng 11/2016?")

[GIN] 2025/06/04 - 08:50:35 | 200 |  2.661588566s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/06/04 - 08:50:35 | 200 |   21.819084ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:35 | 200 |   25.768043ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:35 | 200 |   29.897245ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:35 | 200 |   26.009813ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:35 | 200 |   26.286557ms |       127.0.0.1 | POST     "/api/embed"


time=2025-06-04T08:50:35.385Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:35.385Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:35.385Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:35.385Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
decode: cannot decode batches with this context (use llama_encode() instead)
time=2025-06-04T08:50:35.409Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
time=2025-06-04T08:50:35.409Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:35.409Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:35.409Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
decode: cannot decode batches with this context (use llam

[GIN] 2025/06/04 - 08:50:39 | 200 |  4.234333225s |       127.0.0.1 | POST     "/api/chat"




Sự kiện đã diễn ra vào tháng 11/2016 là **bắt giữ bà Choi Soon-sil** vì những cáo buộc liên quan đến việc can thiệp vào công việc nhà nước và lợi dụng mối quan hệ với Tổng thống Park Geun-hye để gây quỹ bất hợp pháp.

In [18]:
# Ask about what did not happened in the given article
news_qa("Lý Tự Trọng đã làm gì trong bài báo này?")

[GIN] 2025/06/04 - 08:50:43 | 200 |  3.471418581s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/06/04 - 08:50:43 | 200 |   24.650463ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:43 | 200 |   28.684486ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:43 | 200 |   31.472128ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:43 | 200 |   24.198324ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:43 | 200 |   26.969994ms |       127.0.0.1 | POST     "/api/embed"


time=2025-06-04T08:50:43.246Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:43.246Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:43.246Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:43.246Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
decode: cannot decode batches with this context (use llama_encode() instead)
time=2025-06-04T08:50:43.271Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:43.271Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
time=2025-06-04T08:50:43.271Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:43.271Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
decode: cannot decode batches with this context (use llam

[GIN] 2025/06/04 - 08:50:48 | 200 |  4.742097463s |       127.0.0.1 | POST     "/api/chat"




Không có thông tin nào liên quan đến Lý Tự Trọng trong các bài báo được cung cấp. Câu hỏi này dựa trên thông tin không tồn tại trong văn bản đã cho.

In [19]:
# Test Chat Bot conversation memory
news_qa("Câu hỏi trước của tôi là gì?")

[GIN] 2025/06/04 - 08:50:54 | 200 |   6.30135842s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2025/06/04 - 08:50:54 | 200 |   24.172522ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |   26.841305ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |   27.680969ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |    23.53735ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |   23.350411ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |   28.501822ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |   24.685366ms |       127.0.0.1 | POST     "/api/embed"


time=2025-06-04T08:50:54.446Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:54.446Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:54.446Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:54.446Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
decode: cannot decode batches with this context (use llama_encode() instead)
time=2025-06-04T08:50:54.470Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:54.470Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:54.470Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
time=2025-06-04T08:50:54.470Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
decode: cannot decode batches with this context (use llam

[GIN] 2025/06/04 - 08:50:54 | 200 |   25.136782ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |    24.21228ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |   22.812487ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/06/04 - 08:50:54 | 200 |   22.469496ms |       127.0.0.1 | POST     "/api/embed"


time=2025-06-04T08:50:54.661Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
time=2025-06-04T08:50:54.661Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:54.662Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:54.662Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
decode: cannot decode batches with this context (use llama_encode() instead)
time=2025-06-04T08:50:54.689Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat
time=2025-06-04T08:50:54.689Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_eta
time=2025-06-04T08:50:54.689Z level=WARN source=types.go:572 msg="invalid option provided" option=mirostat_tau
time=2025-06-04T08:50:54.689Z level=WARN source=types.go:572 msg="invalid option provided" option=tfs_z
decode: cannot decode batches with this context (use llam

[GIN] 2025/06/04 - 08:51:00 | 200 |  6.099154784s |       127.0.0.1 | POST     "/api/chat"




Câu hỏi trước có thể là: **"Choi Soon-sil đã thừa nhận điều gì trong vụ bê bối chính trị tại Hàn Quốc?"**  
Hoặc một câu hỏi liên quan đến vai trò của bà Choi trong scandal, như: **"Bà Choi Soon-sil có liên quan gì đến việc chỉnh sửa bài phát biểu của Tổng thống Park Geun-hye?"**  
**Lý do:** Document đã nêu rõ Choi thừa nhận đã chỉnh sửa một số bài phát biểu của Park khi cô còn là ứng cử viên tổng thống, và bà là bạn thân của Tổng thống.

## Conclusion
This notebook is about experimenting and implementing RAG for small scale LLMs with under **3 billion parameters** due to my limitation of hardware.  

After implemented **Vector Database** for storing news article's chunk of embeddings to prevent overly long sequence of texts; and **RAG** to retrieve these embeddings effectively, LLMs now can answer questions about given article precisely.  
  
I have also tested on **gemma3-1b** but this model appear to hallucinate when i ask it about Ly Tu Trong (see *Ask about what not happened in the article*). It always associate an action happened in the article with Ly Tu Trong, which is not true.  

**Qwen3-1.7b** on the other hand easily avoid the mistake mentioned above. But still, **both models** have no memory of past question.  

I will treat this as future direction for improvement. 